In [5]:
#pip install tensorflow transformers  tf-keras pandas
from transformers import pipeline
import pandas as pd
import os

In [3]:
#load model
classifier = pipeline("text-classification", 
                      model="j-hartmann/emotion-english-distilroberta-base", 
                      return_all_scores=True)

tf_model.h5:   0%|          | 0.00/329M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/home/ucloud/.local/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [105]:
#load data 
data = pd.read_csv("../in/GoT-scripts/Game_of_Thrones_Script.csv")
#add emotion columns to copy of dataframe


data_out[["anger", "disgust", "fear", "joy", "neutral", "sadness", "suprise"]] = ""

In [107]:
data.index

RangeIndex(start=0, stop=23911, step=1)

In [34]:
for sent in range(len(data["Sentence"])):
    sentence = data["Sentence"][sent]
    emot_sent = classifier(sentence)
    

In [66]:
test = classifier(data["Sentence"][0])

In [100]:
test_2 = pd.DataFrame.from_records(test[0]).T
test_2.columns  = list(test_2.iloc[0])

In [104]:
test_2.appe

label       anger
score    0.094708
Name: anger, dtype: object

In [47]:
pd.DataFrame.from_records(test_out[1:3])
test_out = classifier(list(data["Sentence"][1:5]))

,0,1,2,3,4,5,6
0,"{'label': 'anger', 'score': 0.04903155937790871}","{'label': 'disgust', 'score': 0.01809948869049...","{'label': 'fear', 'score': 0.029709266498684883}","{'label': 'joy', 'score': 0.004798834677785635}","{'label': 'neutral', 'score': 0.16870249807834...","{'label': 'sadness', 'score': 0.01527708675712...","{'label': 'surprise', 'score': 0.7143812775611..."
1,"{'label': 'anger', 'score': 0.019147297367453575}","{'label': 'disgust', 'score': 0.04209727421402...","{'label': 'fear', 'score': 0.005443821661174297}","{'label': 'joy', 'score': 0.005737967789173126}","{'label': 'neutral', 'score': 0.9114251732826233}","{'label': 'sadness', 'score': 0.01093045342713...","{'label': 'surprise', 'score': 0.0052179908379..."
